In [31]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [48]:
from sklearn.metrics import accuracy_score as acs
from sklearn.metrics import confusion_matrix as cm
from matplotlib import pyplot as plt
import itertools

In [9]:
!pip install ml

  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 Proxy Access Denied',))': /simple/ml/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 Proxy Access Denied',))': /simple/ml/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 Proxy Access Denied',))': /simple/ml/
  Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 407 Proxy Access Denied',))': /simple/ml/
  Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection br

In [11]:
dataframe = pd.read_csv("c:/users/administrator/desktop/datasets-v5/task-1/task1.train.txt",delimiter="\t")

In [12]:
dataframe.to_csv("c:/users/administrator/desktop/New_Task1.train.csv", encoding='utf-8', index=False)

In [14]:
articles = pd.read_csv("c:/users/administrator/desktop/New_Task1.train.csv", header=None)

In [76]:
articles.shape

(35986, 2)

In [28]:
np.sum(articles.isna())

Title_Content    0
article_id       0
label            0
dtype: int64

In [54]:
articles.columns =['Title_Content','article_id','label']
articles = articles.set_index('article_id')
articles.head()

,Title_Content,label
article_id,,
727600136,"Et tu, Rhody? A recent editorial in the Provi...",non-propaganda
731714618,A recent post in The Farmington Mirror — our t...,non-propaganda
731714635,"President Donald Trump, as he often does while...",non-propaganda
728627182,"February is Black History Month, and nothing l...",non-propaganda
728627443,"The snow was so heavy, whipped up by gusting w...",non-propaganda


In [157]:
x = articles['Title_Content']
y = articles['label']
y.value_counts()

non-propaganda    31965
propaganda         4021
Name: label, dtype: int64

In [158]:
len(articles[pd.isnull(articles.label)])

0

In [159]:
x_tr,x_ts,y_tr,y_ts = tts(x,y,test_size =0.20)

ModuleNotFoundError: No module named 'smote'

In [172]:
from imblearn.over_sampling import SMOTE
import smote 
import rus

ModuleNotFoundError: No module named 'smote'

# TFIDF

In [164]:
tfidfv = TfidfVectorizer(stop_words='english',encoding ='UTF-8', max_df=0.8)


In [160]:
TfidfVectorizer?

In [165]:
x_bw_m = tfidfv.fit(x_tr)

In [166]:
x_tr_tfm = x_bw_m.transform(x_tr)
x_ts_tfm = x_bw_m.transform(x_ts)

# PA

In [167]:
from sklearn.linear_model import PassiveAggressiveClassifier

pac = PassiveAggressiveClassifier(n_iter=50)

model3 = pac.fit(x_tr_tfm, y_tr)

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [168]:
y_tsp = model3.predict(x_ts_tfm)
confm_ = cm(y_pred=y_tsp,y_true= y_ts)
confm_

array([[6329,   71],
       [ 178,  620]], dtype=int64)

In [170]:
tn,fp,fn,tp = confm_.ravel()

rec = tp/(tp+fn)
pre = tp/(tp+fp)

F1_PA = (2*pre*rec)/(pre+rec)
F1_PA

0.8327736736064472

# MLP_NN	

In [153]:

mlp = MLPClassifier(hidden_layer_sizes=(5,5,4))      # initially(554_0.787)(573_0.556)

model = mlp.fit(x_tr_tfm , y_tr)

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [65]:
y_tsp = mlp.predict(x_ts_tfm)

In [67]:
score= acs(y_ts, y_tsp)
score

0.9532234160800297

In [68]:
tn,fp,fn,tp = cm(y_true=y_ts,y_pred=y_tsp).ravel()
metrics.confusion_matrix(y_true=y_ts,y_pred=y_tsp)

array([[9367,  186],
       [ 319,  924]], dtype=int64)

In [69]:
rec = (tp/(tp+fn))             # recall/sensitivity
pre = (tp/(tp+fp))             # precision
f1_mlp = (2*pre*rec)/(pre+rec)

print(f1_mlp)

0.7853803654908627


In [75]:
results  = pd.DataFrame()

In [90]:
#results.head()

# KERAS

In [92]:
from keras.models import Sequential
from keras import layers

dim = x_tr_tfm.shape[1]  # Number of features

sq = Sequential()
sq.add(layers.Dense(10, input_dim=dim, activation='relu'))
sq.add(layers.Dense(1, activation='sigmoid'))

In [93]:
sq.compile(loss='binary_crossentropy', 
             optimizer='adam', 
              metrics=['accuracy'])
sq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                1539210   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,539,221
Trainable params: 1,539,221
Non-trainable params: 0
_________________________________________________________________


In [103]:
d = {'non-propaganda':0,'propaganda':1}

y_tr1 = y_tr.map(d)
y_ts1 = y_ts.map(d)

In [107]:
model_fit = sq.fit(x_tr_tfm, y_tr1, epochs=2,verbose=False, validation_data=(x_ts_tfm, y_ts1),batch_size=10)

In [108]:
loss, accuracy = sq.evaluate(x_tr_tfm, y_tr1, verbose=False)

print("Training Accuracy: {:.4f}".format(accuracy))


loss, accuracy = sq.evaluate(x_ts_tfm, y_ts1, verbose=False)

print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9999
Testing Accuracy:  0.9617


In [124]:
y_tsp = sq.predict_classes(x_ts_tfm)

In [126]:
#tn,fp,fn,tp = cm(y_ts1,y_tsp).ravel()

type(y_ts1),type(y_tsp)


(pandas.core.series.Series, numpy.ndarray)

In [127]:
cm(y_ts1,y_tsp)

array([[7916,   84],
       [ 261,  736]], dtype=int64)

In [132]:
tn,fp,fn,tp = cm(y_ts1,y_tsp).ravel()
tn,fp,fn,tp

(7916, 84, 261, 736)

In [131]:
rec = (tp/(tp+fn))             # recall/sensitivity
pre = (tp/(tp+fp))             # precision
f1_k = (2*pre*rec)/(pre+rec)

print(f1_k)

0.8101265822784809


In [135]:
from sklearn.metrics import classification_report as cf

rep = cf(y_ts1,y_tsp)
rep

'             precision    recall  f1-score   support\n\n          0       0.97      0.99      0.98      8000\n          1       0.90      0.74      0.81       997\n\navg / total       0.96      0.96      0.96      8997\n'

# KERAS 2

In [137]:
dim = x_tr_tfm.shape[1]  # Number of features

sq2 = Sequential()
sq2.add(layers.Dense(10, input_dim=dim, activation='relu'))
sq2.add(layers.Dense(1, activation='sigmoid'))

sq2.compile(loss='binary_crossentropy', 
             optimizer='adam', 
              metrics=['accuracy'])
sq2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 10)                1539210   
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 1,539,221
Trainable params: 1,539,221
Non-trainable params: 0
_________________________________________________________________


In [146]:
model_fit1 = sq2.fit(x_tr_tfm, y_tr1, epochs=20,verbose=False, validation_data=(x_ts_tfm, y_ts1),batch_size=20)

In [147]:
loss, accuracy = sq2.evaluate(x_tr_tfm, y_tr1, verbose=False)

print("Training Accuracy: {:.4f}".format(accuracy))


loss, accuracy = sq2.evaluate(x_ts_tfm, y_ts1, verbose=False)

print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 1.0000
Testing Accuracy:  0.9631


In [148]:
y_tsp = sq.predict_classes(x_ts_tfm)

In [149]:
cm(y_ts1,y_tsp)

array([[7916,   84],
       [ 261,  736]], dtype=int64)

In [150]:
tn,fp,fn,tp = cm(y_ts1,y_tsp).ravel()
tn,fp,fn,tp

(7916, 84, 261, 736)

In [151]:
rec = (tp/(tp+fn))             # recall/sensitivity
pre = (tp/(tp+fp))             # precision
f1_k = (2*pre*rec)/(pre+rec)

print(f1_k)

0.8101265822784809


In [152]:
rep = cf(y_ts1,y_tsp)
rep

'             precision    recall  f1-score   support\n\n          0       0.97      0.99      0.98      8000\n          1       0.90      0.74      0.81       997\n\navg / total       0.96      0.96      0.96      8997\n'